## Loading preprosed data

In [ ]:
LEVEL="sequence"

In [ ]:
PATH = "../weights/auto_encoder{}/version_1".format(LEVEL)

In [3]:
BATCH_SIZE=256
LAMBDA = 10
NUM_EPOCH = 1000000

In [4]:
NUM_OF_ACIDS = 21
EMBEDDING_SIZE = 8

In [5]:
import tensorflow as tf
tf.__version__

/home/donatasrep/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


'1.6.0'

In [6]:
import numpy as np
train_data = np.load("..//data//train_features_3.6.1.7LLevel_4.npy")
val_data = np.load("..//data//val_features_3.6.1.7LLevel_4.npy")

In [7]:
train_data.shape, val_data.shape

((271, 120), (0,))

In [8]:
SEQUENCE_LENGTH=train_data.shape[1]
SEQUENCE_LENGTH

120

In [9]:
STEPS_PER_EPOCH = int(train_data.shape[0]/BATCH_SIZE)+1
STEPS_PER_EPOCH

2

# Model

## Discriminator

In [10]:
def discriminator(x, is_training):

    with tf.variable_scope('discriminator', reuse=tf.AUTO_REUSE) as scope:
        print('discriminator')
        conv1 = tf.layers.conv2d(
            inputs=x,
            filters=64,
            kernel_size=[3,EMBEDDING_SIZE],
            strides=(2,1),
            padding="same",
            activation=tf.nn.leaky_relu,
            name = "dconv1")
        print(conv1.shape)
        # Convolutional Layer #2
        conv2 = tf.layers.conv2d(
            inputs=conv1,
            filters=128,
            kernel_size=[3,EMBEDDING_SIZE],
            strides=(2,1),
            padding="same",
            activation=tf.nn.leaky_relu,
            name = "dconv2")
        conv2 = tf.layers.batch_normalization(conv2, name = "dbn1")
        print(conv2.shape)
        conv3 = tf.layers.conv2d(
            inputs=conv2,
            filters=256,
            kernel_size=[3,EMBEDDING_SIZE],
            strides=(2,1),
            padding="same",
            activation=tf.nn.leaky_relu,
            name = "dconv3")
        conv3 = tf.layers.batch_normalization(conv3, name = "dbn2")
        print(conv3.shape)
        flat = tf.layers.flatten(conv3, name="dflat")
        output = tf.layers.dense(inputs=flat,
                                 activation=None,
                                 units=1,
                                 name="doutput")
        print(output.shape)
        output = tf.reshape(output, [-1])
        print(output.shape)
        return output

# Generator

In [ ]:
import math
def generator(input_batch=None, is_training=True):
    with tf.variable_scope('generator') as scope:
        print('generator')
        if input_batch is None:
            input_batch = tf.random_normal([BATCH_SIZE, 128])
        dim = math.floor(SEQUENCE_LENGTH/4)
        print (input_batch.shape)   
        dense1 = tf.layers.dense(inputs=input_batch,
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=0.02),
                                 bias_initializer=tf.zeros_initializer (),
                                 units=dim*EMBEDDING_SIZE*256,
                                 activation=tf.nn.relu,
                                 name="dense1")
        reshaped1 = tf.reshape(dense1, shape=[-1, -1, 1, 1], name='reshape1')
        reshaped1 = tf.layers.batch_normalization(reshaped1, name = "gbn1")
        print(reshaped1.shape)
        up1 = tf.keras.layers.UpSampling2D(size=(2, 1))(reshaped1)
        conv2 = tf.layers.conv2d(inputs=up1, 
                                 filters=128,
                                 kernel_size=[3,EMBEDDING_SIZE],
                                 padding="same",
                                 activation=tf.nn.relu,
                                 name = "conv2")
        conv2 = tf.layers.batch_normalization(conv2, name = "gbn2")
        print(conv2.shape)
        up2 = tf.keras.layers.UpSampling2D(size=(2, 1))(conv2)
        conv3 = tf.layers.conv2d(inputs=up2, 
                                 filters=64,
                                 kernel_size=[3,EMBEDDING_SIZE],
                                 padding="same",
                                 activation=tf.nn.relu,
                                 name = "conv3")
        conv3 = tf.layers.batch_normalization(conv3, name = "gbn3")
        print(conv3.shape)         
        conv4 = tf.layers.conv2d(inputs=conv3, 
                                 filters=1,
                                 kernel_size=[3,EMBEDDING_SIZE],
                                 padding="same",
                                 activation=tf.nn.sigmoid,
                                 name = "conv4")    
        print(conv4.shape)
        return conv4

## Graph

In [12]:
tf.reset_default_graph()

with tf.variable_scope('input'):
    real_sequences = tf.placeholder(tf.int32, [None, SEQUENCE_LENGTH], name='real_sequence')
    is_training = tf.placeholder(tf.bool, name='is_train')

dataset = tf.data.Dataset.from_tensor_slices(real_sequences)
dataset = dataset.shuffle(buffer_size=10000, reshuffle_each_iteration=True)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE)).repeat(NUM_EPOCH)
iterator = dataset.make_initializable_iterator()

acid_embeddings = tf.get_variable("acid_embeddings", [NUM_OF_ACIDS, EMBEDDING_SIZE])

batch_real_sequences = iterator.get_next()

embedded_real_sequences = tf.nn.embedding_lookup(acid_embeddings, batch_real_sequences)
embedded_real_sequences = tf.reshape(embedded_real_sequences, shape=[-1, SEQUENCE_LENGTH, EMBEDDING_SIZE, 1], name='embedded_real_sequences')

In [13]:
fake = generator(is_training=is_training)
logits_real = discriminator(embedded_real_sequences, is_training)
logits_fake = discriminator(fake, is_training)
d_loss = tf.reduce_mean(logits_fake) - tf.reduce_mean(logits_real) # This optimizes the discriminator.
g_loss = -tf.reduce_mean(logits_fake)  # This optimizes the generator.

# # wgan-gp gradient panelty 
with tf.name_scope("Gradient_penalty"):
    eps = tf.random_uniform([BATCH_SIZE,1, 1, 1], minval=0.0,maxval=1.0)
    interpolates = embedded_real_sequences + eps*(fake - embedded_real_sequences)

    gradients = tf.gradients(discriminator(interpolates, is_training), [interpolates])[0]
    slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
    gradient_penalty = tf.reduce_mean(tf.square(slopes - 1.))
    d_loss += 10 * gradient_penalty
    
D_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,'discriminator')
G_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'generator')

trainer_d = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.5, beta2=0.9).minimize(d_loss, var_list=D_vars)
trainer_g = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.5, beta2=0.9).minimize(g_loss, var_list=G_vars)

generator
(256, 128)
(256, 30, 8, 256)
(256, 60, 8, 128)
(256, 120, 8, 64)
(256, 120, 8, 1)
discriminator
(256, 60, 8, 64)
(256, 30, 8, 128)
(256, 15, 8, 256)
(256, 1)
(256,)
discriminator
(256, 60, 8, 64)
(256, 30, 8, 128)
(256, 15, 8, 256)
(256, 1)
(256,)
discriminator
(256, 60, 8, 64)
(256, 30, 8, 128)
(256, 15, 8, 256)
(256, 1)
(256,)


## Helpers for training model

## Review generated examples

In [14]:
 def save_weights(saver, sess, path):
    save_path = saver.save(sess, path)
    print("Model saved in path: %s" % save_path)

In [15]:
def mean(l):
    if len(l) == 0:
        return 0
    else:
        return sum(l) / float(len(l))

In [16]:
def print_summary(steps, dLosses, gLosses):    
    if steps % int(STEPS_PER_EPOCH*100) == 0:
        print('steps:{} \td_loss:{:.4f} \tg_loss:{:.4f}'.format(steps, mean(dLosses), mean(gLosses)))
        dLosses, gLosses = [], [] 
    return dLosses, gLosses

In [18]:
def display_sequence():
    sequences = reverse_embedding_lookup(acid_embeddings, tf.squeeze(fake))
    generated_sequences, logits = sess.run([sequences, logits_fake], feed_dict={is_training: False})
    indexToLetter= {0: '0', 1: 'A', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'K', 10: 'L', 11: 'M', 12: 'N', 
                13: 'P', 14: 'Q', 15: 'R', 16: 'S', 17: 'T', 18: 'V', 19: 'W', 20: 'Y'}
    best_sequence = "".join([ indexToLetter[acid_index] for acid_index in generated_sequences[np.argmax(logits)]]) 
    worst_sequence = "".join([ indexToLetter[acid_index] for acid_index in generated_sequences[np.argmin(logits)]]) 
    print("{} | Discriminator value {}".format(best_sequence, logits[np.argmax(logits)]))
    print("{} | Discriminator value {}".format(worst_sequence, logits[np.argmin(logits)]))

In [19]:
import datetime
def save_model(saver, sess):
    # Epoch ended
    if steps % (STEPS_PER_EPOCH*100) == 0:
        display_sequence()
        print("Epoch {}. Fineshed at {}".format((steps/STEPS_PER_EPOCH), str(datetime.datetime.now()).split('.')[0]))
        save_weights(saver, sess, PATH)

## Running model

In [20]:
sess = tf.Session()
saver = tf.train.Saver(max_to_keep=3)
sess.run(tf.global_variables_initializer())

# CAUTION: Training the model

In [23]:
print ("Start training with batch size: {}, epoch num: {}".format(BATCH_SIZE, NUM_EPOCH))
sess.run(iterator.initializer, feed_dict={real_sequences: train_data})
steps, gen_iterations = 0, 0
dLosses, gLosses = [], [] 
while True:
    try:
        d_iters = (100 if gen_iterations < 25 or gen_iterations % 500 == 0 else 5)
        for k in range(d_iters): # Discriminator
            _, dLoss = sess.run([trainer_d, d_loss], feed_dict={is_training: True})
            steps = steps + 1
            dLosses.append(dLoss)
            dLosses, gLosses = print_summary(steps, dLosses, gLosses)
            save_model(saver, sess)

        # Generator
        _, gLoss = sess.run([trainer_g, g_loss], feed_dict={is_training: True})
        gLosses.append(gLoss)
        steps = steps + 1
        gen_iterations = gen_iterations + 1
        dLosses, gLosses = print_summary(steps, dLosses, gLosses)
        save_model(saver, sess)
    except tf.errors.OutOfRangeError:
        print ("Training is finished")
        break;            

Start training with batch size: 256, epoch num: 1000000
steps:200 	d_loss:-24.5621 	g_loss:25.3325
MFSYFSIIIIYEEFIIFEFFESS0FFEMSSSEEFRFSPFERGEGIFGEFFEEIISSSSSSESEESEFSFIEFEESSESFEFYPFIFFFFSIIYFFFFFFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -25.818758010864258
MSSFSMSFYEYEIIFEFRFFEERIFSSSGGSEYPEFMFFISFIIEFEISEIIYSEFMSFFSMSSSFSSFISPSSRIFRSFFSFESEISSSFYSFFIIFSFFFFFFFFFFFFFFFFFFFFF | Discriminator value -26.824750900268555
Epoch 100.0. Fineshed at 2018-04-29 06:04:36
Model saved in path: ../weights/wgan_sequence/version_6
steps:400 	d_loss:-24.5519 	g_loss:25.6990
MFFFSIEEYFIIIEESIEEGFFMFSSRSRFYESESEEFFFFSPFERGEFFFIEMREFCEESYESSEEEFSPGRFFSIIFFWSFFREIIEEFEERFEFFEEIFFFFFFFFFFFFFFFFFFF | Discriminator value -26.015729904174805
MSSFRRFFSISISFEIIEYRFFIFFEFEFMESEGESRSSESFSFRISFSEESMEFFEFISSFIIGPEEFEFESISFESEEYFFSERFFSY0FFFFFFFFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -26.906044006347656
Epoch 200.0. Fineshed at 2018-04-29 06:05:25
Model saved in path: ../weights/wgan_sequence/version_6
step

Model saved in path: ../weights/wgan_sequence/version_6
steps:3800 	d_loss:-24.5060 	g_loss:24.0900
MSIRFYFSISESFIEFFFRFSSSEIFIESGYSEFIRF0FFERFEFCFSGESEFFIFSIFYCREEPEMFMFSEFFYSIIEFEEEFSRFSFPYFFFFFFFFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -23.87957191467285
MSEEIIYSRYFFSFRFIFFEFRSSEEEEFSESEYEFSFFFFFFFEFMFYEESEEFSISEESSYSESEEFEGEEEEIEESIFEEISFFRFFSSFFFFFFFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -24.687694549560547
Epoch 1900.0. Fineshed at 2018-04-29 06:19:09
Model saved in path: ../weights/wgan_sequence/version_6
steps:4000 	d_loss:-24.5164 	g_loss:24.8600
MPSESYSFIFEFEEEIFEFEFRSFERCRFSESESSEFFFFMFFIRFSFFFIEPRFFCISSSISSIGFSEEIRFFSFFFFSSISEEIIEEFSERFF0FFFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -24.961938858032227
MSSESMSFSISYIIGYFRFFFCEEFRESIISIEFFFFPSFYFYEFFSGEEFFSCSFFEFISSEEMFSSFESFSSEIMSGSFEFESFEFSSFFSFFYIISFFFFFFFFFFFFFFFFFFFFF | Discriminator value -25.696165084838867
Epoch 2000.0. Fineshed at 2018-04-29 06:19:56
Model saved in path: ../weights/wgan_sequence/version_6
s

Model saved in path: ../weights/wgan_sequence/version_6
steps:7400 	d_loss:-24.5395 	g_loss:28.8634
MSRYEMSFFIRIEIFIRPIFGIFRMFSEFEFRSIEFIISGSFSSSFEFSEIFIEEEMSFFSMSSFSSSIISPSSRIFRSFFSFESEISSSEFSFFSIRFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -28.85667610168457
MSFEGEESSSFEIEFEFFEFEFRFMSRFSFCESRSSFRFFFFEEFEFFFEGEEMSSESYYESISSEEEFFEIERSREFIEEEEFFFSFFRY0FFFFFFFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -29.639999389648438
Epoch 3700.0. Fineshed at 2018-04-29 06:33:16
Model saved in path: ../weights/wgan_sequence/version_6
steps:7600 	d_loss:-24.5344 	g_loss:29.4405
MISFEYFFSISFIEFFFRFSERIRFSE0FSEEYFEFMFFFSFFIMFEGEFFFEFIFIEEEFSFFFMSSSSSSIESPSSRIFRSYFSFESEISS0FF0FFSSRFFFFFFFFFFFFFFFFFF | Discriminator value -29.45011329650879
MYSYSFFISEIFIFFEFRFSESFYFFFSEISEYFRFIFFESFEISFMFFEESSFEFYSFYRSGRMFRSFFEFSIYEEFEISEEFFFSYFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF | Discriminator value -30.175785064697266
Epoch 3800.0. Fineshed at 2018-04-29 06:34:03
Model saved in path: ../weights/wgan_sequence/version_6
st

KeyboardInterrupt: 

## Validation of discriminator

In [17]:
def reverse_embedding_lookup(acid_embeddings, embedded_sequence):
    acid_embeddings_expanded = tf.tile(tf.expand_dims(acid_embeddings, axis = 0), [BATCH_SIZE, 1,1])
    emb_distances = tf.matmul(
        tf.nn.l2_normalize(acid_embeddings_expanded, axis=1),
        tf.nn.l2_normalize(embedded_sequence, axis=1),
        transpose_b=True)
    return tf.argmax(emb_distances, axis=1)

In [ ]:
val_real = discriminator(real_reshaped, is_training, reuse=True)
val_fake = discriminator(embedded_random_sequences, is_training, reuse=True)
val_loss = tf.reduce_mean(val_real-val_fake)
real_predictions = tf.rint(val_real)
fake_predictions = tf.rint(val_fake)
correct_real_predictions = tf.equal(real_predictions, tf.zeros([BATCH_SIZE], dtype=tf.float32))
correct_fake_predictions = tf.equal(fake_predictions, tf.ones([BATCH_SIZE], dtype=tf.float32))
casted_real = tf.cast(correct_real_predictions, tf.float32)
casted_fake = tf.cast(correct_fake_predictions, tf.float32)
accuracy = (tf.reduce_mean(casted_real) + tf.reduce_mean(casted_fake))/2

In [ ]:
#Validate discriminator by giving from validate data set and randomly generated
print ('validating discriminator...')
sess.run(iterator.initializer, 
         feed_dict={real_sequences: val_data, random_sequences: get_random_sequence(val_data.shape[0])})
losses = []
accuracies = []
while True:
    try:
        v_loss, v_accuracy = sess.run([val_loss, accuracy], feed_dict={is_training: False})
        losses.append(v_loss)
        accuracies.append(v_accuracy)
    except tf.errors.OutOfRangeError:
        print ('Validation g_loss:{:.4f} ,accuracy :{:.4f}'.format(mean(losses), mean(accuracies)))
        break

In [21]:
def restore_weights(saver, sess, path):
    saver.restore(sess, path)
    print("Model restored.")

In [22]:
restore_weights(saver, sess, PATH)

INFO:tensorflow:Restoring parameters from ../weights/wgan_sequence/version_6
Model restored.


## Review generated examples

In [45]:
sequences = reverse_embedding_lookup(acid_embeddings, tf.squeeze(fake))
print (sequences.shape)

(64, 48)


In [141]:
print ('Generating sequences...')
generated_sequences = sess.run([sequences], feed_dict={is_training: False})
generated_sequences[0]

Generating sequences...


array([[15,  7, 17, ..., 15, 15, 15],
       [15, 15, 15, ...,  7, 17, 15],
       [15, 15, 18, ..., 15, 17, 15],
       ...,
       [15, 15, 15, ..., 17, 11, 15],
       [17, 17, 15, ...,  5, 17, 11],
       [17, 17, 15, ..., 17,  6, 15]])

In [142]:
indexToLetter= {0: '0', 1: 'A', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'K', 10: 'L', 11: 'M', 12: 'N', 
                13: 'P', 14: 'Q', 15: 'R', 16: 'S', 17: 'T', 18: 'V', 19: 'W', 20: 'Y'}
print("".join([ indexToLetter[acid_index] for acid_index in s ]))

In [143]:
for s in generated_sequences[0]:
    print("".join([ indexToLetter[acid_index] for acid_index in s ]))
    print("")
    print("".join([ indexToLetter[acid_index] for acid_index in s if acid_index != 0 ]))
    print("---------------------")

RHTRRRVTTTTMTRRRRRTRTHTTRRRRRRRTHRMTTMMRTTRTTRRR

RHTRRRVTTTTMTRRRRRTRTHTTRRRRRRRTHRMTTMMRTTRTTRRR
---------------------
RRRMRRHRRRTRTTVRWTHTRHRRCRRRRRHTTHRRRRTTTRRVRHTR

RRRMRRHRRRTRTTVRWTHTRHRRCRRRRRHTTHRRRRTTTRRVRHTR
---------------------
RRVRRRTTHRRHRTYRTRTMTRRMRVRRFRTMRRRRHRTRTTTTRRTR

RRVRRRTTHRRHRTYRTRTMTRRMRVRRFRTMRRRRHRTRTTTTRRTR
---------------------
RRMRRRVTRRRRRRFVRTRMTRRTTTTHMRRTMTVTRRRTRRRRRTRR

RRMRRRVTRRRRRRFVRTRMTRRTTTTHMRRTMTVTRRRTRRRRRTRR
---------------------
RRTMRFRTTTYTTTMTGRRTTTTRTMTTRRRRRRRRVRRRTRVTTRTT

RRTMRFRTTTYTTTMTGRRTTTTRTMTTRRRRRRRRVRRRTRVTTRTT
---------------------
MRRRRTRRTRVMVVRMTRTRRRVRRRRRTFHRRRRRRRTRVTRRRRTR

MRRRRTRRTRVMVVRMTRTRRRVRRRRRTFHRRRRRRRTRVTRRRRTR
---------------------
TRRRRVRRRRTRTRRRRRRTTHRTRTTRRMRTRRVVITTRRRFRRRRR

TRRRRVRRRRTRTRRRRRRTTHRTRTTRRMRTRRVVITTRRRFRRRRR
---------------------
MTTTRRRFRRRTGRRTRRTFVRTTTTRTRRRVRHTRTRRTRRRTTMRR

MTTTRRRFRRRTGRRTRRTFVRTTTTRTRRRVRHTRTRRTRRRTTMRR
---------------------
RTRRTVTTHRTRTTRTTRTRTRTRRMRTMTFH